In [4]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import json
import io
import csv
import shutil
from tqdm import tqdm
from collections import defaultdict

In [2]:
#进行数据清洗的测试，从训练集中读取数据
data = pd.read_csv('train_usual.csv',header=None)
#加入列名
data.columns=["label","text"]

In [3]:
data.head()

,label,text
0,1,气死 姐姐 了 ， 快二是 阵亡 了 吗 ， 尼玛 ， 一个半 小时 过去 了 也 没 上车
1,2,妞妞 啊 ， 今天 又 承办 了 一个 发文 登记 文号 是 126 ~ 嘻 ~ 么 么 哒...
2,6,这里 还 值得注意 另 一个 事实 ， 就是 张鞠存 原有 一个 东溪 草堂 为 其 读书处 。
3,6,这 在 前 华约 国家 ( 尤其 是 东德 ) 使用 R - 73 的 首次 联合演习 期间...
4,5,TinyThief 上 wii 了 ？ ！


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27766 entries, 0 to 27765
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   27766 non-null  int64 
 1   text    27766 non-null  object
dtypes: int64(1), object(1)
memory usage: 434.0+ KB


In [5]:
data['label'].value_counts()

1    8343
6    5749
2    5378
3    4990
5    2086
4    1220
Name: label, dtype: int64

In [6]:
data.describe()

,label
count,27766.000000
mean,3.020709
std,1.898212
min,1.000000
25%,1.000000
50%,3.000000
75%,5.000000
max,6.000000


In [7]:
# 查看每一列是否有缺失：
data.isna().any(axis=0)
# 查看每一行是否有缺失：
data.isna().any(axis=1)
# 查看所有数据中是否有缺少最快的，没有输出False，反之为True：
data.isna().values.any()
#data[data.isna().values==True]

False

In [8]:
data['text'].map(len).describe()
#计算文本的长度，其中mean为均值

count    27766.000000
mean        69.081142
std         48.949356
min         13.000000
25%         35.000000
50%         52.000000
75%         84.000000
max        371.000000
Name: text, dtype: float64

In [9]:
#data['label'].value_counts(normalize=True).plot(kind='bar');
data['label'].value_counts()

1    8343
6    5749
2    5378
3    4990
5    2086
4    1220
Name: label, dtype: int64

In [10]:
data = pd.read_csv('test_usual.csv',header=None)
data.columns=["label","text"]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5000 non-null   int64 
 1   text    5000 non-null   object
dtypes: int64(1), object(1)
memory usage: 78.2+ KB


In [11]:
data = pd.read_csv('dev_usual.csv',header=None)
data.columns=["label","text"]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   2000 non-null   int64 
 1   text    2000 non-null   object
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


In [12]:
#定义数据清洗函数
def clean_data(old_path,save_path,save_name):
    with open(old_path, 'r', encoding='utf8') as f:
        data = defaultdict(list)
        texts = f.readlines()
        for line in tqdm(texts, desc=old_path):
            label,text= line.strip().split(',',1)
            emoji = json.load(open('C:/Users/23106/Desktop/weibopy/emoji.json', 'r', encoding='utf8'))
            for emoji, emoji_text in emoji.items():
                text = text.replace(emoji,emoji_text )
            new_text=re.sub('[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~\s]+', "", text)
            data['text'].append(new_text)
            
            data['label'].append(label)
            #将label与text交换位置
        df = pd.DataFrame(data)
        df.to_csv(os.path.join(save_path,save_name), index=False,encoding='utf8', header=False, sep='\t')
    return

In [13]:
clean_data('C:/Users/23106/Desktop/weibopy/train_usual.csv','C:/Users/23106/Desktop/weibopy/new/','train.csv')
clean_data('C:/Users/23106/Desktop/weibopy/test_usual.csv','C:/Users/23106/Desktop/weibopy/new/','test.csv')
clean_data('C:/Users/23106/Desktop/weibopy/dev_usual.csv','C:/Users/23106/Desktop/weibopy/new/','valid.csv')

C:/Users/23106/Desktop/weibopy/dev_usual.csv: 100%|██████████| 2000/2000 [00:01<00:00, 1360.11it/s]


In [14]:
#从经过数据清洗后生成的文件中读取数据
data = pd.read_csv('C:/Users/23106/Desktop/weibopy/new/train.csv',sep='\t',header=None)
#加入列名
data.columns=["text","label"]

In [15]:
data.head(15)

,text,label
0,气死姐姐了快二是阵亡了吗尼玛一个半小时过去了也没上车,1
1,妞妞啊今天又承办了一个发文登记文号是嘻么么哒晚安哟,2
2,这里还值得注意另一个事实就是张鞠存原有一个东溪草堂为其读书处,6
3,这在前华约国家尤其是东德使用的首次联合演习期间被一些北约组织的飞行员所证实,6
4,上了,5
5,每天都以紧张的心情工作真的是太累太不放松了想要爆发一下,1
6,语文军数学军英语军物理军政治军历史军生物军地理军八科联军侵犯我班我班战败被迫签订家长会条约赔...,1
7,我不是一个优秀的演员不能微笑着旁观你们幸福,3
8,当你变优秀时你想要的都会来找你,2
9,累了一天会宿舍听下我搞基新歌在看看我段宜恩美图心都被治愈了,2


In [16]:
#将label中的数字进行转化，便于查看
data.loc[data['label']==1, 'label'] = '愤怒'
data.loc[data['label']==2, 'label'] = '积极'
data.loc[data['label']==3, 'label'] = '悲伤'
data.loc[data['label']==4, 'label'] = '恐惧'
data.loc[data['label']==5, 'label'] = '惊奇'
data.loc[data['label']==6, 'label'] = '无情绪'

In [17]:
data.head(14)

,text,label
0,气死姐姐了快二是阵亡了吗尼玛一个半小时过去了也没上车,愤怒
1,妞妞啊今天又承办了一个发文登记文号是嘻么么哒晚安哟,积极
2,这里还值得注意另一个事实就是张鞠存原有一个东溪草堂为其读书处,无情绪
3,这在前华约国家尤其是东德使用的首次联合演习期间被一些北约组织的飞行员所证实,无情绪
4,上了,惊奇
5,每天都以紧张的心情工作真的是太累太不放松了想要爆发一下,愤怒
6,语文军数学军英语军物理军政治军历史军生物军地理军八科联军侵犯我班我班战败被迫签订家长会条约赔...,愤怒
7,我不是一个优秀的演员不能微笑着旁观你们幸福,悲伤
8,当你变优秀时你想要的都会来找你,积极
9,累了一天会宿舍听下我搞基新歌在看看我段宜恩美图心都被治愈了,积极


In [18]:
data = data[['text', 'label']]
data.to_csv('C:/Users/23106/Desktop/weibopy/new/train.csv', index=False, sep="\t")

In [19]:
#从经过数据清洗后生成的文件中读取数据
data = pd.read_csv('C:/Users/23106/Desktop/weibopy/new/test.csv',sep='\t',header=None)
#加入列名
data.columns=["text","label"]
#将label中的数字进行转化，便于查看
data.loc[data['label']==1, 'label'] = '愤怒'
data.loc[data['label']==2, 'label'] = '积极'
data.loc[data['label']==3, 'label'] = '悲伤'
data.loc[data['label']==4, 'label'] = '恐惧'
data.loc[data['label']==5, 'label'] = '惊奇'
data.loc[data['label']==6, 'label'] = '无情绪'
data = data[['text', 'label']]
data.to_csv('C:/Users/23106/Desktop/weibopy/new/test.csv', index=False, sep="\t")

In [20]:
#从经过数据清洗后生成的文件中读取数据
data = pd.read_csv('C:/Users/23106/Desktop/weibopy/new/valid.csv',sep='\t',header=None)
#加入列名
data.columns=["text","label"]
#将label中的数字进行转化，便于查看
data.loc[data['label']==1, 'label'] = '愤怒'
data.loc[data['label']==2, 'label'] = '积极'
data.loc[data['label']==3, 'label'] = '悲伤'
data.loc[data['label']==4, 'label'] = '恐惧'
data.loc[data['label']==5, 'label'] = '惊奇'
data.loc[data['label']==6, 'label'] = '无情绪'
data = data[['text', 'label']]
data.to_csv('C:/Users/23106/Desktop/weibopy/new/valid.csv', index=False, sep="\t")

In [21]:
#读取数据
train_data = pd.read_csv('C:/Users/23106/Desktop/weibopy/new/train.csv',sep='\t',header=None)
train_data.columns=["label","text"]
test_data = pd.read_csv('C:/Users/23106/Desktop/weibopy/new/test.csv',sep='\t',header=None)
test_data.columns=["label","text"]

In [22]:
test_data.head()

,label,text
0,text,label
1,宿舍要民汉合宿了为毛都大三了还要折腾我,愤怒
2,早上的我竟然也变成了一个无理取闹的人,悲伤
3,多年来周天先生率智多星的律师策划师团队走出巴蜀挺进海西在厦门特区的发祥地安营扎寨,无情绪
4,听故事的时候你总喜欢眼巴巴的问后来呢可是当后来自己成为讲故事的人才发现后来故事和话语就在嘴边...,悲伤


In [23]:
train_data.head()

,label,text
0,text,label
1,气死姐姐了快二是阵亡了吗尼玛一个半小时过去了也没上车,愤怒
2,妞妞啊今天又承办了一个发文登记文号是嘻么么哒晚安哟,积极
3,这里还值得注意另一个事实就是张鞠存原有一个东溪草堂为其读书处,无情绪
4,这在前华约国家尤其是东德使用的首次联合演习期间被一些北约组织的飞行员所证实,无情绪


In [24]:
#查看测试集信息
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5000 non-null   object
 1   text    5001 non-null   object
dtypes: object(2)
memory usage: 78.3+ KB


In [25]:
#构建情感分析模型
import paddle

In [26]:
import paddlehub as hub

In [27]:
#检测环境是否正常
paddle.utils.run_check()

Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle works well on 1 GPUs.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


In [28]:
label_list=list(data.label.unique())
print(label_list)

['愤怒', '积极', '悲伤', '无情绪', '恐惧', '惊奇']


In [29]:
label_map = { 
    idx: label_text for idx, label_text in enumerate(label_list)
}
print(label_map)

{0: '愤怒', 1: '积极', 2: '悲伤', 3: '无情绪', 4: '恐惧', 5: '惊奇'}


In [30]:
# 此处选择ernie_tiny预训练模型并设置微调任务为6分类任务
model = hub.Module(name="ernie_tiny", task='seq-cls', num_classes=6, label_map=label_map)

[2022-05-24 19:25:52,291] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\ernie_tiny.pdparams


In [47]:
from paddlehub.datasets.base_nlp_dataset import InputExample, TextClassificationDataset

In [48]:
import os, io, csv

In [49]:
print(label_list)

['愤怒', '积极', '悲伤', '无情绪', '恐惧', '惊奇']


In [50]:
DIR="C:/Users/23106/Desktop/weibopy/new/"

In [51]:
class TEXTALS(TextClassificationDataset):
    def __init__(self, tokenizer, mode='train', max_seq_len=128):
        if mode == 'train':
            data_file = 'train.csv'  # 训练集
        elif mode == 'test':
            data_file = 'test.csv'   # 测试集
        else:
            data_file = 'valid.csv'  # 验证集
        
        super(TEXTALS, self).__init__(
            base_path=DIR,
            data_file=data_file,
            tokenizer=tokenizer,
            max_seq_len=max_seq_len,
            mode=mode,
            is_file_with_header=True,
            label_list=label_list
            )

    # 解析文本文件里的样本
    def _read_file(self, input_file, is_file_with_header: bool = False):
        if not os.path.exists(input_file):
            raise RuntimeError("The file {} is not found.".format(input_file))
        else:
            with io.open(input_file, "r", encoding="UTF-8") as f:
                reader = csv.reader(f, delimiter="\t")
                examples = []
                seq_id = 0
                header = next(reader) if is_file_with_header else None
                for line in reader:
                    try:
                        example = InputExample(guid=seq_id, text_a=line[0], label=line[1])
                        seq_id += 1
                        examples.append(example)
                    except:
                        continue
                return examples
                
train_dataset = TEXTALS(model.get_tokenizer(), mode='train', max_seq_len=128)
test_dataset = TEXTALS(model.get_tokenizer(), mode='test', max_seq_len=128)
valid_dataset = TEXTALS(model.get_tokenizer(), mode='valid', max_seq_len=128)

# 查看训练集前3条
for e in train_dataset.examples[:3]:
    print(e)
# 查看验证集前3条
for e in test_dataset.examples[:3]:
    print(e)
# 查看测试集前3条
for e in valid_dataset.examples[:3]:
    print(e)

[2022-05-24 20:56:33,219] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\vocab.txt
[2022-05-24 20:56:33,221] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\spm_cased_simp_sampled.model
[2022-05-24 20:56:33,222] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\dict.wordseg.pickle
[2022-05-24 20:56:41,891] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\vocab.txt
[2022-05-24 20:56:41,892] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\spm_cased_simp_sampled.model
[2022-05-24 20:56:41,893] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\dict.wordseg.pickle
[2022-05-24 20:56:46,015] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\vocab.txt
[2022-05-24 20:56:46,016] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\spm_cased_simp_sampled.model
[2022-05-24 20:56:46,017] [    INFO] - Already cached C:\Us

text=气死姐姐了快二是阵亡了吗尼玛一个半小时过去了也没上车	label=愤怒
text=妞妞啊今天又承办了一个发文登记文号是嘻么么哒晚安哟	label=积极
text=这里还值得注意另一个事实就是张鞠存原有一个东溪草堂为其读书处	label=无情绪
text=宿舍要民汉合宿了为毛都大三了还要折腾我	label=愤怒
text=早上的我竟然也变成了一个无理取闹的人	label=悲伤
text=多年来周天先生率智多星的律师策划师团队走出巴蜀挺进海西在厦门特区的发祥地安营扎寨	label=无情绪
text=所以注定我这辈子是做不了商人妈蛋	label=愤怒
text=无论是心情多么低沉的夜晚天光大亮后都是崭新的开始	label=积极
text=帽子怎么就变绿色幸好只是试一下	label=悲伤


In [36]:
#选择优化策略和运行配置
optimizer = paddle.optimizer.AdamW(learning_rate=4e-5, parameters=model.parameters())
trainer = hub.Trainer(model, optimizer, checkpoint_dir='./ckpt', use_gpu=True, use_vdl=True)

[2022-05-24 19:26:10,675] [ WARNING] - PaddleHub model checkpoint not found, start from scratch...


In [37]:
print(paddle.device.get_device())

gpu:0


In [38]:
trainer.train(train_dataset, epochs=6, batch_size=32, eval_dataset=valid_dataset, save_interval=1)

[2022-05-24 19:26:16,874] [   TRAIN] - Epoch=1/6, Step=10/868 loss=1.5888 acc=0.3344 lr=0.000040 step/sec=1.62 | ETA 00:53:32
[2022-05-24 19:26:22,821] [   TRAIN] - Epoch=1/6, Step=20/868 loss=1.3246 acc=0.5469 lr=0.000040 step/sec=1.68 | ETA 00:52:35
[2022-05-24 19:26:28,869] [   TRAIN] - Epoch=1/6, Step=30/868 loss=1.1497 acc=0.6000 lr=0.000040 step/sec=1.65 | ETA 00:52:33
[2022-05-24 19:26:34,813] [   TRAIN] - Epoch=1/6, Step=40/868 loss=1.1003 acc=0.6031 lr=0.000040 step/sec=1.68 | ETA 00:52:18
[2022-05-24 19:26:40,747] [   TRAIN] - Epoch=1/6, Step=50/868 loss=0.8621 acc=0.7031 lr=0.000040 step/sec=1.69 | ETA 00:52:09
[2022-05-24 19:26:46,827] [   TRAIN] - Epoch=1/6, Step=60/868 loss=0.8665 acc=0.6687 lr=0.000040 step/sec=1.64 | ETA 00:52:15
[2022-05-24 19:26:52,931] [   TRAIN] - Epoch=1/6, Step=70/868 loss=0.8537 acc=0.7000 lr=0.000040 step/sec=1.64 | ETA 00:52:21
[2022-05-24 19:26:59,068] [   TRAIN] - Epoch=1/6, Step=80/868 loss=0.8787 acc=0.7031 lr=0.000040 step/sec=1.63 | ETA 0

KeyboardInterrupt: 

In [ ]:
# 在测试集上评估当前训练模型
result = trainer.evaluate(test_dataset, batch_size=32)

In [ ]:
data = [
    # 悲伤
    ["你也不用说对不起,只是若相惜"],
    # 积极
    ["幸福其实很简单"],
    # 恐惧
    ["恐惧感啊。生病"],
    # 积极
    ["待你长发及腰,我们一起耕耘时光。我愿等待"]
]
label_list=['愤怒', '积极', '悲伤', '无情绪', '恐惧', '惊奇']
label_map = {
    idx: label_text for idx, label_text in enumerate(label_list)
}
model = hub.Module(
    name='ernie_tiny',
    task='seq-cls',
    num_classes=6,
    load_checkpoint='./ckpt/best_model/model.pdparams',
    label_map=label_map)
results = model.predict(data, max_seq_len=128, batch_size=32, use_gpu=True)
for idx, text in enumerate(data):
    print('Data: {} \t Lable: {}'.format(text[0], results[idx]))

In [67]:
import numpy as np

In [68]:
from sklearn.metrics import precision_recall_fscore_support,f1_score,precision_score,recall_score

In [74]:
df = pd.read_csv('C:/Users/23106/Desktop/weibopy/new/test.csv',sep = '\t')
#print(df)
df.info()
news1 = pd.DataFrame(columns=['label'])
news1['label'] = df["label"]
news = pd.DataFrame(columns=['text_a'])
news['text_a'] = df["text_a"]
# 首先将pandas读取的数据转化为array
data_array = np.array(news)
#print(data_array)
# 然后转化为list形式
data_list =data_array.tolist()
print(data_list)
data_array1 = np.array(news1)
y_val =data_array1.tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text_a  4999 non-null   object
 1   label   4999 non-null   object
dtypes: object(2)
memory usage: 78.2+ KB
[['宿舍要民汉合宿了为毛都大三了还要折腾我'], ['早上的我竟然也变成了一个无理取闹的人'], ['多年来周天先生率智多星的律师策划师团队走出巴蜀挺进海西在厦门特区的发祥地安营扎寨'], ['听故事的时候你总喜欢眼巴巴的问后来呢可是当后来自己成为讲故事的人才发现后来故事和话语就在嘴边后来眼泪几度泪凝于睫是真的说不下去了'], ['汉代前人口不多而晋末宋末唐末及中国历史上三次移民潮给皖南徽州送来了大量人口人口众多山多地少怎么办'], ['伊川基地伊川生产基地坐落于世界著名文化遗产洛阳龙门石窟南公里处'], ['明太祖洪武十年（年）改变圜丘礼制定每年孟春正月合祀天地于南郊建大祀殿以圜形大屋覆盖祭坛'], ['睡得好好的要去别的学校听课天啊'], ['管理学真是水的一比努力的想听依然坚持不过一分钟考研怎么办呀'], ['想去日本台湾冰岛英国北欧签证说小短假也说'], ['为何这么晚了还那么有精神呢明天还要上班呢能乖乖心情么'], ['知道去年的分就能去中传我快炸了'], ['我觉得在地铁里蹭别人的电视剧看而且还笑的那么夸张的人简直太可爱了噗哈哈哈'], ['昨晚得知竟把牙的阴历生日给忘了先是说不出后又蒙出个初还信誓旦旦的说对就初十'], ['做完饭不洗锅什么都往水池扔什么都往厕所倒所以三天两头都是堵不讲卫生也就罢了天天带男人回家带男人回家也就罢了天天摔桌子吵架嗓门大如泼妇堪比河东狮吼二房东这是招的什么人'], ['舍友推荐的中韩梦之队确实好看看到了最新一期的紧张激烈的篮球赛紧张到我只希望中国赢一直很喜欢韩国的但看见韩国教练在中国比分落后时咧个大嘴笑我真是讨厌这个破教练'], ['开刷泰国恐怖片惊恐

In [75]:
import paddlehub as hub
label_list=['愤怒', '积极', '悲伤', '无情绪', '恐惧', '惊奇']
label_map = {
    idx: label_text for idx, label_text in enumerate(label_list)
}
model = hub.Module(
    name='ernie_tiny',
    task='seq-cls',
    num_classes=6,
    load_checkpoint='./ckpt/best_model/model.pdparams',
    label_map=label_map)

[2022-05-24 21:09:42,865] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\ernie_tiny.pdparams
[2022-05-24 21:09:45,622] [    INFO] - Loaded parameters from C:\Users\23106\Desktop\weibopy\ckpt\best_model\model.pdparams


In [76]:
y_pre = model.predict(data_list, max_seq_len=128, batch_size=32, use_gpu=True)




[2022-05-24 21:09:46,471] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\vocab.txt
[2022-05-24 21:09:46,473] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\spm_cased_simp_sampled.model
[2022-05-24 21:09:46,474] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\dict.wordseg.pickle


In [77]:
f1 = f1_score(y_val, y_pre, average='macro')
p = precision_score(y_val, y_pre, average='macro')
r = recall_score(y_val, y_pre, average='macro')
print(f1, p, r)

0.7402975243969186 0.7317424403773524 0.7517300369557383
